In [1]:
import json
from pprint import pprint
import h5py
import scipy.io
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import np_utils
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2

Using TensorFlow backend.


In [2]:
def get_image(data,data_in,CATEGORIES):
    img = data[data_in].get('filename','')
    DATADIR = "sg_dataset"
    
    for category in CATEGORIES:
        path = os.path.join(DATADIR,category)
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_COLOR)
        break
    return img_array

In [3]:
def get_object(data,data_in,rel_in):
    ob1 = data[data_in].get('relationships',)[rel_in].get('objects','')[0]
    ob2 = data[data_in].get('relationships',)[rel_in].get('objects','')[1]
    return ob1,ob2

In [4]:
def get_coordinate(data,data_in,ob1,ob2):
    y1 = data[data_in].get('objects','')[ob1].get('bbox','').get('y','')
    x1 = data[data_in].get('objects','')[ob1].get('bbox','').get('x','')
    w1 = data[data_in].get('objects','')[ob1].get('bbox','').get('w','')
    h1 = data[data_in].get('objects','')[ob1].get('bbox','').get('h','')

    y2 = data[data_in].get('objects','')[ob2].get('bbox','').get('y','')
    x2 = data[data_in].get('objects','')[ob2].get('bbox','').get('x','')
    w2 = data[data_in].get('objects','')[ob2].get('bbox','').get('w','')
    h2 = data[data_in].get('objects','')[ob2].get('bbox','').get('h','')
    return x1,y1,w1,h1,x2,y2,w2,h2

In [5]:
def UnionBox(x1,x2,y1,y2,h1,h2,w1,w2):
    ub_x = 0
    ub_y = 0
    ub_w = 0
    ub_h = 0

    if x1 > x2:
        ub_x = x2
    else:
        ub_x = x1

    if y1 > y2:
        ub_y = y2
    else:
        ub_y = y1

    if x1+w1 > x2+w2:
        ub_w = x1+w1-ub_x
    else:
        ub_w = x2+w2-ub_x

    if y1+h1 > y2+h2:
        ub_h = y1+h1-ub_y
    else:
        ub_h = y2+h2-ub_y
        
    return ub_x,ub_y,ub_w,ub_h

In [6]:
def crop_image(img_arr,ub_y,ub_x,ub_w,ub_h):
    #crop the image with the union box and return it
    #Use this for Union_WB_B method to black out the exterior
    crop_img = img_arr[int(ub_y):int(ub_y+ub_h), int(ub_x):int(ub_x+ub_w)].copy()
    return crop_img

In [7]:
def check_rel(rel):
    #predicats = ['above','behind','beneath','left of','right of',
    #             'below','has','hold','in','inside','in front of','near',
    #             'next to','on','on the left of','on the right of',
    #             'under','wear','at','has on','adjacent to','is behind the','is to left of',
    #             'is right of the','']
    
    spatial  = ['above','behind','beneath','below','has',
                'in','front','near','next','top',
                 'on','left','right','under']
    rel1 = rel
    for i in spatial:
        if rel.find(i) == 0:
            #print(rel1)
            #print(i)
            rel = i
            return rel
            #print("FAILURE ",rel1)
            
        #    print ("Found",rel)
    #rel = None
    #if rel not in spatial:
    #    rel = None
    rel = None
    return rel

#Objects aren't needed only saving relationships not training object classifier
def check_obj(obj1,obj2):
    objs = ['airplane','bag','ball','basket','bear','bed','bench',
            'bike','boat','bottle','bowl','box','building','bus',
            'bush','cabinet','camera','can','car','cart','cat','chair',
            'clock','coat','computer','cone','counter','cup','desk','dog',
            'elephant','engine','face','faucet','giraffe','glasses','grass',
            'hand','hat','helmet','horse','hydrant','jacket','jeans','keyboard',
            'kite','lamp','laptop','luggage','monitor','motorcycle','mountain',
            'mouse','oven','pants','paper','person','phone','pillow','pizza',
            'plane','plant','plate','post','pot','ramp','refrigerator','road',
            'roof','sand','shelf','shirt','shoe','shoes','shorts','sink','skateboard',
            'skis','sky','snowboard','sofa','stove','street',
            'suitcase','sunglasses','surfboard','table','tie',
            'tower','traffic light','train','trash can','tree',
            'trees','truck','umbrella','van','vase','watch','wheel']
    
    #if obj1 not in objs:
    #    obj1 = None
    
    #if obj2 not in objs:
    #    obj2 = None
    
    return obj1,obj2

In [8]:
def write_to_file(data,data_in,rel_in,crop_img,new_dir):
    
    ob1_name = data[data_in].get('relationships',)[rel_in].get('text','')[0]
    rel = data[data_in].get('relationships',)[rel_in].get('text','')[1]
    ob2_name = data[data_in].get('relationships',)[rel_in].get('text','')[2]
    rel = check_rel(rel)
    #ob1_name,ob2_name = check_obj(ob1_name,ob2_name)
    
    if rel != None:
        image_name = data[data_in].get('filename',)[:-4]+"_"+ob1_name+"_"+rel+"_"+ob2_name+"0"+".jpg"
        name = rel
        #Name directory after the label
        #name = rel
        dirName = new_dir+name
        try:
            # Create target Directory
            os.mkdir(dirName)
            #print("Directory " , dirName ,  " Created ") 
        except FileExistsError:
            pass

        filename = dirName+"/"+image_name.replace(" ", "")
        exists = False
        i = int(filename[len(filename)-5:-4])
        while exists != True:
            if (os.path.isfile(filename) != True):
                cv2.imwrite(filename,crop_img)
                exists = True
            else:
                i += 1
                filename = filename[:-5]+str(i)+".jpg"

In [9]:
def Union_WB_B(img_array,x1,y1,h1,w1,x2,y2,h2,w2):
    
    img_array[0:img_array.shape[0] , 0:img_array.shape[1]] = (0,0,0)
    img_array[int(y1):int(y1+h1), int(x1):int(x1+w1)] = (0,255,0)
    img_array[int(y2):int(y2+h2), int(x2):int(x2+w2)] = (255,0,0)
    
    return img_array

def Union_WB(img_array,x1,y1,h1,w1,x2,y2,h2,w2):
    obj1 = img_array[int(y1):int(y1+h1), int(x1):int(x1+w1),:].copy()
    obj2 = img_array[int(y2):int(y2+h2), int(x2):int(x2+w2),:].copy()

    #temp_array = img_array.copy()
    temp_array = np.zeros(img_array.shape, np.uint8)
    #temp_array[int(y1):int(y1+h1), int(x1):int(x1+w1),:] = obj1
    #temp_array[:,:,:] = img_array[0:img_array.shape[0] , 0:img_array.shape[1],:]
    temp_array[int(y1):int(y1+h1), int(x1):int(x1+w1),:] = img_array[int(y1):int(y1+h1), int(x1):int(x1+w1),:].copy()
    temp_array[int(y2):int(y2+h2), int(x2):int(x2+w2),:] = img_array[int(y2):int(y2+h2), int(x2):int(x2+w2),:].copy()

    return temp_array

In [10]:
def execute_images(method,data,CATEGORIES):
    #Extracting the required data
    for i in range(len(data)):
        img_array = get_image(data,i,CATEGORIES)
        for j in range(len(data[i].get('relationships',))):
            try:
                obj1 , obj2 = get_object(data,i,j)
                x1,y1,w1,h1,x2,y2,w2,h2 = get_coordinate(data,i,obj1,obj2)
                ub_x,ub_y,ub_w,ub_h = UnionBox(x1,x2,y1,y2,h1,h2,w1,w2)
                
                if method is "Union_WB_B":
                    img_array1 = Union_WB_B(img_array.copy(),x1,y1,h1,w1,x2,y2,h2,w2)
                    crop_img = crop_image(img_array1,ub_y,ub_x,ub_w,ub_h)
                    crop_img = cv2.resize(crop_img, (224, 224))
                    write_to_file(data,i,j,crop_img,"Union_WB_B/")
                elif method is "Union":
                    crop_img = crop_image(img_array,ub_y,ub_x,ub_w,ub_h)
                    crop_img = cv2.resize(crop_img, (224, 224))
                    write_to_file(data,i,j,crop_img,"Union/")
                elif method is "Union_WB":
                    img_array1 = Union_WB(img_array.copy(),x1,y1,h1,w1,x2,y2,h2,w2)
                    crop_img = crop_image(img_array1,ub_y,ub_x,ub_w,ub_h)
                    crop_img = cv2.resize(crop_img, (224, 224))
                    write_to_file(data,i,j,crop_img,"Union_WB/") 
                else:
                    print("ERROR")
                    break
            except:
                pass

In [61]:
with open('sg_dataset/sg_train_annotations.json') as f:
    data = json.load(f)
    
execute_images("Union",data,["sg_train_images"])
execute_images("Union_WB",data,["sg_train_images"])
execute_images("Union_WB_B",data,["sg_train_images"])

with open('sg_dataset/sg_test_annotations.json') as f1:
    data1 = json.load(f1)

execute_images("Union",data1,["sg_test_images"])
execute_images("Union_WB",data1,["sg_test_images"])
execute_images("Union_WB_B",data1,["sg_test_images"])

In [12]:
from imutils import paths
import random
import cv2
import os

def write_cvs(dataset):
    # grab all image paths and create a training and testing split
    imagePaths = sorted(list(paths.list_images(dataset)))
    random.shuffle(imagePaths)
    i = int(len(imagePaths) * 0.20)
    j = int(len(imagePaths) * 0.60)
    
    trainPaths = imagePaths[:j]
    testPaths = imagePaths[j:j+i]
    validPaths = imagePaths[j+i:]
    
    csv_path_training = dataset+"_training.csv"
    csv_path_testing = dataset+"_testing.csv"
    csv_path_validation = dataset+"_validation.csv"

    #define the datasets
    datasets = [
        ("training", trainPaths, csv_path_training),
        ("testing", testPaths, csv_path_testing),
        ("validation", validPaths, csv_path_validation)
    ]
    
    # loop over the data splits
    for (dType, imagePaths, outputPath) in datasets:
        # open the output CSV file for writing
        print("[INFO] building '{}' split...".format(dType))
        f = open(outputPath, "w")

        # loop over all input images
        for imagePath in imagePaths:
            try:
                # load the input image and resize it to 64x64 pixels
                image = cv2.imread(imagePath)
                image = cv2.resize(image, (224, 224))

                # create a flattened list of pixel values
                image = [str(x) for x in image.flatten()]

                # extract the class label from the file path and write the
                # label along pixels list to disk
                label = imagePath.split(os.path.sep)[-2].split("_")
                label = label[0]+','+label[1]
                f.write("{},{}\n".format(label, ",".join(image)))
            except:
                pass

        # close the output CSV file
        f.close()


In [85]:
write_cvs("Union")
write_cvs("Union_WB")
write_cvs("Union_WB_B")

[INFO] building 'training' split...
[INFO] building 'testing' split...
[INFO] building 'validation' split...
[INFO] building 'training' split...
[INFO] building 'testing' split...
[INFO] building 'validation' split...
[INFO] building 'training' split...
[INFO] building 'testing' split...
[INFO] building 'validation' split...


In [13]:
#Writing clothes set
write_cvs("datasetClothes")

[INFO] building 'training' split...
[INFO] building 'testing' split...
[INFO] building 'validation' split...
